<a href="https://colab.research.google.com/github/m10k1/gemma_cookbook/blob/main/Gemma_Basics_with_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma Basics (Hugging Face)

このノートブックでGemmaモデルのロードの仕方、ファインチューニングの方法をデモします。

## セットアップ

このチュートリアルを実行するに亜h、Colabの実行環境が必要です。
今回の場合T4GPUを使用します。

1. Colabratoryの画面のメニュー「ランタイム」を選びます。
2. 「ランタイムのタイプの変更」を選択
3. ハードウェアアクセラレータで「T4」を選択

## Gemma セットアップ

チュートリアルに入る前にGemmaをセットアップします。
1. Hugging faceのアカウント： Hugging faceのアカウントがない場合はあかんとを作成します。
2. Gemmaモデルのアクセス：Gemmeモデルのページに行き、使用条件を受け入れます。
3. Colab with Gemma Power: このチュートリアルを実行するには、Gemma２を扱うことができるランタイムを選択します。
4. Hugging Face Token：Hugging Faceのアクセストークンを用意します。writeパーミッション。

## HFトークンを設定する

Hugging　FaceのトークンをColabのシークレットマネージャーに保存します。



In [7]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

## 依存ライブラリをインストール

依存するライブラリをインストールします

In [5]:
! pip install --upgrade -q transformers huggingface_hub peft accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Hugging Face Hubにログインする


In [8]:
from huggingface_hub import login

login(os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Gemma 2B モデルをインスタンス化

Gemma は軽量なモデルであり、GoogleによるSotaなオープンモデルです。Geminiモデルと同じ研究から作られました。Text to Text、デコーダーオンリーLLMです。英語で利用可能であり、オープンウェイトで、事前訓練されたバリアントとインストラクションチューニングされたバリアントがあります。
Gemmaモデルはさまざまなテキスト生成タスクに合うように作られています。QA、要約、理由付けなど。比較的小さなサイズであるため、ノートパソコンやデスクトップ、独自のクラウドインフラストラクチャなど、リソースが限られた環境でも導入が可能で、最先端のAIモデルへのアクセスを民主化し、誰もがイノベーションを促進できるようにする。

それでは、Hugging Face Hubからモデルをロードしましょう

In [20]:
model_id = "google/gemma-1.1-2b-it"
device = "cuda:0"

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)


In [22]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ウェイトを減らすために量子化する
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# 最終的なモデルをロードする
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 試してみる

In [24]:
prompt = "My favorite color is"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=20)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)


My favorite color is blue. It represents calmness, trust, and serenity. It brings me a sense of peace and tranquility


In [25]:
prompt = "What can you use an LLM for? Answer:"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=512)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

#

What can you use an LLM for? Answer:

**An LLM (Large Language Model) can be used for a wide range of tasks, including:**

* **Information retrieval:** Providing summaries, answering questions, and providing factual information.
* **Content creation:** Generating creative text formats, writing different kinds of content, and translating languages.
* **Summarization:** Extracting key points from large amounts of text.
* **Code generation:** Assisting developers in writing code and debugging errors.
* **Customer service:** Providing personalized and contextual support to users.
* **Education:** Providing personalized learning experiences and generating educational materials.
* **Marketing:** Creating targeted marketing campaigns and analyzing customer data.
* **Translation:** Translating documents and websites between multiple languages.
* **Creative writing:** Generating original and imaginative content.


## LoRAを使ってファインチューニングする

このセクションでは、Large Language Model (LLM)をトレーニングして、有名な引用文を生成することに焦点を当てます。 ここでは、有名な作家、哲学者、指導者のような出力を生成できるようにモデルを微調整するプロセスを探ります。


In [26]:
# ファインチューニングする前の出力を試す
text = "Quote: Imagination is more"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Quote: Imagination is more than just a spark of genius; it is the fertile ground from which great art, science, and'

In [27]:
# データセットを読み込む
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
print("Example item:", data["train"][0])

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Example item: {'quote': '“Be yourself; everyone else is already taken.”', 'author': 'Oscar Wilde', 'tags': ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']}


In [28]:
# 引用をトークン化する
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [29]:
from peft import LoraConfig

# Define tuning parameters
lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [30]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}<eos>"
    return [text]

In [31]:
import transformers
from trl import SFTTrainer

# Create Trainer objects that takes care of the process
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:381: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [32]:
# Let's run the fine-tuning
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: matzmtok to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,7.706000
2,6.188900
3,6.708200
4,6.498600
5,6.126700
6,5.064000
7,5.178800
8,4.307500
9,5.583900
10,4.887400


TrainOutput(global_step=10, training_loss=5.82500958442688, metrics={'train_runtime': 86.5262, 'train_samples_per_second': 0.462, 'train_steps_per_second': 0.116, 'total_flos': 16634596884480.0, 'train_loss': 5.82500958442688})

In [33]:
# Testing the models after fine-tuning
text = "Quote: Imagination is"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is the faculty of the mind to invent, to conceive, to invent, and to realize things that do


## モデルをHugging Face Hubにプッシュする

Hugging Faceでは訓練したモデルをハブに簡単に保存できる


In [34]:
# Note: The token needs to have "write" permisssion
#       You can chceck it here:
#       https://huggingface.co/settings/tokens
model.push_to_hub("my-gemma-2-finetuned-model")

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/maztmok/my-gemma-2-finetuned-model/commit/67c9a338a6b1db9f25d57cda8f84d48be5835f8b', commit_message='Upload GemmaForCausalLM', commit_description='', oid='67c9a338a6b1db9f25d57cda8f84d48be5835f8b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/maztmok/my-gemma-2-finetuned-model', endpoint='https://huggingface.co', repo_type='model', repo_id='maztmok/my-gemma-2-finetuned-model'), pr_revision=None, pr_num=None)

## テキスト生成推論（TGI）を使ってモデルを提供する

Text Generation Inferenceは、Gemmaのような大規模言語モデル（LLM）の導入と使用を簡素化するツールキットです。 テキスト生成タスクのためにモデルを最適化し、より高速に実行し、より早く結果を出せるようにする。 TGIは、複数のグラフィックカード（GPU）に作業負荷を分散して処理を高速化するテンソル並列や、テキスト生成用に特別に設計された最適化コードなどの技術によってこれを実現します。 さらに、TGIは、モデルのパフォーマンスを監視するための分散トレース、詳細なデータ収集のためのPrometheusメトリクス、モデルの出力を保護するための透かしのようなセキュリティ対策など、生産環境に適した機能を提供します。 TGIについての詳細は、公式ドキュメントを参照してください。

TGIでモデルを展開するには、以下の方法があります：

１. **ローカルにデプロイします（Dockerが必要です）：** ローカルマシンでモデルを実行するには、以下のコードセルのコメントを外します。 この方法では、Dockerがインストールされ、GPUが接続されている必要があります。

2. **GKEを使用してGoogle Cloud Platformにデプロイする：**このガイドに従って、Hugging Face TGIを使用してGKE上でGPUを使用するServe GemmaオープンモデルをGoogle CloudのCKEサービスにデプロイします。 このオプションは、高性能推論のためにGPUを活用します。

どちらのデプロイ方法でも、リクエストを送信し、モデルからのテキスト生成レスポンスを受信するためのHTTPエンドポイントが提供されます。

In [35]:
!model="google/gemma-1.1-2b-it" # ID of the model in Hugging Face hube
# (you can use your own fine-tuned model from
# the prevous step)
!volume=$PWD/data               # Shared directory with the Docker container
# to avoid downloading weights every run

# !docker run --gpus all --shm-size 1g -p 8080:80 \
#     -v $volume:/data ghcr.io/huggingface/text-generation-inference:2.0.3 \
#     --model-id $model